# Astronaut Activities in SQL - Exploratory Data Analysis

The goal of this project is to look at the data about astronauts' extravehicular activities.
How many rocks did we bring back from the space?
Which astronaut spent the most time in space?
Which NASA program has cumulatively spent the most hours in space?

Let's find out!

[Source of the data](https://www.kaggle.com/datasets/alexandrepetit881234/astronauts-extra-vehicular-activities)


Disclaimer:
This project is available in Jupiter without using `!pip install ipython-sql` and `%sql` because I did it in the Datacamp cloud solution which allows changing cells from text to Python or SQL.




<div>
<img src="pexels-pixabay-39896.jpg" width="600"/>
</div>



## Exploring our data
Let's start by looking at the table we will be working with.

In [15]:
SELECT *
FROM evas
LIMIT 10;

,date,country,vehicle,duration,crew,purpose,year,program
0,1965-06-03,USA,Gemini IV,36,Ed White,First U.S. EVA. Used HHMU and took photos. G...,1965,Gemini
1,1966-03-16,USA,Gemini VIII,0,David Scott,HHMU EVA cancelled before starting by stuck on...,1966,Gemini
2,1966-06-05,USA,Gemini IX-A,127,Eugene Cernan,"Inadequate restraints, stiff 25ft umbilical an...",1966,Gemini
3,1966-07-19,USA,Gemini X,50,Mike Collins,Standup EVA. UV photos of stars. Ended by ey...,1966,Gemini
4,1966-07-20,USA,Gemini X,39,Mike Collins,Retrieved MMOD experiment from docked Agena. ...,1966,Gemini
5,1966-09-13,USA,Gemini XI,44,Richard Gordon,Attached tether between Agena and Gemini. EVA...,1966,Gemini
6,1966-09-14,USA,Gemini XI,130,Richard Gordon,Standup EVA. Took star photos. Agena tether ops,1966,Gemini
7,1966-11-12,USA,Gemini XII,149,Buzz Aldrin,Standup EVA. Science tasks. Took star photos,1966,Gemini
8,1966-11-13,USA,Gemini XII,126,Buzz Aldrin,Attached tether between Agena and Gemini. UV ...,1966,Gemini
9,1966-11-14,USA,Gemini XII,55,Buzz Aldrin,Standup EVA. Jettisoned equipment. Took photos,1966,Gemini


`purpose` column looks interesting and confusing. Let's take a deeper look at it.

In [16]:
SELECT purpose
FROM evas;

,purpose
0,First U.S. EVA. Used HHMU and took photos. G...
1,HHMU EVA cancelled before starting by stuck on...
2,"Inadequate restraints, stiff 25ft umbilical an..."
3,Standup EVA. UV photos of stars. Ended by ey...
4,Retrieved MMOD experiment from docked Agena. ...
...,...
370,1 hr late start due to airlock valve. Relocat...
371,"Installed plasma experiment/cables/probes, rep..."
372,Power cable clamps installed and Kurs tested i...
373,"Â Installed VINOSLIVOST experiment on MRM2, 2..."


Seems like a lot of different information gathered here.

## What are the most common types of EVAs?
I will generate features to get a rough idea of the most popular types of EVA astronauts.



In [17]:
SELECT
	purpose,
	CASE WHEN purpose ILIKE '%photos%' THEN 1 ELSE 0 END as photography,
	CASE WHEN purpose ILIKE '%collect%' THEN 1 ELSE 0 END as collection,
	CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE '%assemb%' THEN 1 ELSE 0 END as construction,
	CASE WHEN purpose ILIKE '%replace%' or purpose ILIKE '%fix%' or purpose ILIKE '%repair%' or purpose ILIKE '%servic%' THEN 1 ELSE 0 END as repair
FROM evas

,purpose,photography,collection,construction,repair
0,First U.S. EVA. Used HHMU and took photos. G...,1,0,0,0
1,HHMU EVA cancelled before starting by stuck on...,0,0,0,0
2,"Inadequate restraints, stiff 25ft umbilical an...",0,0,0,0
3,Standup EVA. UV photos of stars. Ended by ey...,1,0,0,0
4,Retrieved MMOD experiment from docked Agena. ...,0,0,0,0
...,...,...,...,...,...
370,1 hr late start due to airlock valve. Relocat...,0,0,1,1
371,"Installed plasma experiment/cables/probes, rep...",0,0,1,1
372,Power cable clamps installed and Kurs tested i...,0,0,1,1
373,"Â Installed VINOSLIVOST experiment on MRM2, 2...",0,0,1,0


We are now ready to build this into a final query!

In [18]:
WITH purpose AS(
	SELECT
		purpose,
		CASE WHEN purpose ILIKE '%photos%' THEN 1 ELSE 0 END as photography,
		CASE WHEN purpose ILIKE '%collect%' THEN 1 ELSE 0 END as collection,
		CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE '%assemb%' THEN 1 ELSE 0 END as construction,
		CASE WHEN purpose ILIKE '%replace%' or purpose ILIKE '%fix%' or purpose ILIKE '%repair%' or purpose ILIKE '%servic%' THEN 1 ELSE 0 END as repair
	FROM evas
)

SELECT
	SUM(photography) as count,
	'photography' as type
FROM purpose

UNION

SELECT
	SUM(collection) as count,
	'collection' as type
FROM purpose

UNION

SELECT
	SUM(construction) as count,
	'construction' as type
FROM purpose

UNION

SELECT
	SUM(repair) as count,
	'repair' as type
FROM purpose

ORDER BY count DESC

,count,type
0,191,construction
1,129,repair
2,16,collection
3,13,photography


## How much material has been extracted from EVAs?
Skimming through the `purpose` column, we also saw numerous references to extracting rock/dust or geological material. In this case, it will be difficult to extract the total quantity across the columns. Regular expressions to the rescue!

In [19]:
SELECT
	purpose
FROM evas
WHERE purpose ILIKE '%geologic%' or purpose ILIKE '%rock%';

,purpose
0,First to walk on the moon. Some trouble getti...
1,Collected 75.6 lb of geologic material. ALSEP...
2,Collected 94.4 lb of geologic material. ALSEP...
3,Collected 169 lb of geologic material. ALSEP ...
4,Collected 208 lb of rock/dust (41lb this day)....
5,Collected 82 lb of rock/dust. Drove rover 11.5 km
6,Collected 90 lb of rock/dust. Drove rover 27....
7,Collected 243 lb of geologic material. ALSEP ...


Okay, we now know that the format of the pounds extracted is always `number lbs of rock/geologic`. We can construct a pattern to detect this and extract the number!

In [20]:
SELECT
	purpose,
	SUBSTRING(purpose, '(\d+\.?\d*) lb of ((rock)|(geologic))') AS weight
FROM evas

,purpose,weight
0,First U.S. EVA. Used HHMU and took photos. G...,NaN
1,HHMU EVA cancelled before starting by stuck on...,NaN
2,"Inadequate restraints, stiff 25ft umbilical an...",NaN
3,Standup EVA. UV photos of stars. Ended by ey...,NaN
4,Retrieved MMOD experiment from docked Agena. ...,NaN
...,...,...
370,1 hr late start due to airlock valve. Relocat...,NaN
371,"Installed plasma experiment/cables/probes, rep...",NaN
372,Power cable clamps installed and Kurs tested i...,NaN
373,"Â Installed VINOSLIVOST experiment on MRM2, 2...",NaN


Now we can calculate the total amount!

In [21]:
WITH weights AS(
	SELECT
		purpose,
		SUBSTRING(purpose, '(\d+\.?\d*) lb of ((rock)|(geologic))') AS weight
	FROM evas
)

SELECT SUM(weight::decimal)
FROM weights



,sum
0,1008.3


## Which astronauts have the most time in EVAs?
First, let's see what the maximum count of astronauts in an EVA has been by splitting the `crew` column. 

In [22]:
SELECT
	crew,
	SPLIT_PART(crew, ',', 4) AS first_asutronaut

FROM evas
WHERE SPLIT_PART(crew, ',', 4) != ''

,crew,first_asutronaut


Now we know how we can extract the time for each astronaut! Let's create a CTE we can then use to piece together `duration` information for each astronaut.

In [23]:
WITH austronauts_split AS(
	SELECT
		crew,
		SPLIT_PART(crew, ',', 1) AS first_asutronaut,
		SPLIT_PART(crew, ',', 2) AS second_asutronaut,
		SPLIT_PART(crew, ',', 3) AS third_asutronaut,
		duration
	FROM evas
)

SELECT *
FROM austronauts_split

,crew,first_asutronaut,second_asutronaut,third_asutronaut,duration
0,Ed White,Ed White,,,36
1,David Scott,David Scott,,,0
2,Eugene Cernan,Eugene Cernan,,,127
3,Mike Collins,Mike Collins,,,50
4,Mike Collins,Mike Collins,,,39
...,...,...,...,...,...
370,"Gennady Padalka, Yuri Malenchenko",Gennady Padalka,Yuri Malenchenko,,351
371,"Pavel Vinogradov, Roman Romanenko",Pavel Vinogradov,Roman Romanenko,,398
372,"Fyodor Yurchikhin, Alexander Misurkin",Fyodor Yurchikhin,Alexander Misurkin,,394
373,"Fyodor Yurchikhin, Alexander Misurkin",Fyodor Yurchikhin,Alexander Misurkin,,449


Now it's just a matter of splitting apart this table and appending each set of results to one table.

In [24]:
WITH austronauts_split AS(
	SELECT
		crew,
		SPLIT_PART(crew, ',', 1) AS first_asutronaut,
		SPLIT_PART(crew, ',', 2) AS second_asutronaut,
		SPLIT_PART(crew, ',', 3) AS third_asutronaut,
		duration
	FROM evas
),

austronaut_durations AS(
	SELECT 
		first_asutronaut AS astronaut,
		duration
	FROM austronauts_split
	WHERE first_asutronaut != ''
	
	UNION ALL
	
	SELECT 
		second_asutronaut AS astronaut,
		duration
	FROM austronauts_split
	WHERE second_asutronaut != ''
	
	UNION ALL
	
	SELECT 
		third_asutronaut AS astronaut,
		duration
	FROM austronauts_split
	WHERE third_asutronaut != ''
)

SELECT
	astronaut,
	SUM(duration) as total_duration
FROM austronaut_durations
GROUP BY astronaut
ORDER BY total_duration desc
limit 10;

,astronaut,total_duration
0,Jerry Ross,3501
1,Anatoly Solovyev,3086
2,Scott Parazynski,2825
3,Nikola Budarin,2672
4,John Grunsfeld,2527
5,Mike Lopez-Alegria,2501
6,Mike Fincke,2472
7,Dan Tani,2351
8,Victor Afanasyev,2314
9,Rick Mastracchio,2311


## What is the cumulative amount of time spent in EVAs over time?
Finally, let's take a look at the cumulative time spent in EVAs by year and space program. To do so, we will need to use a window function.

In [25]:
SELECT
	TO_DATE(year::text, 'YYYY') as year,
	program,
	duration,
	SUM(duration) OVER(PARTITION BY program ORDER BY year) AS cumulative_duration
FROM (
	SELECT
		year,
		program,
		SUM(duration) as duration
	FROM evas
	GROUP BY 1,2
	ORDER BY 1,2
) as sum
ORDER BY 1,2




,year,program,duration,cumulative_duration
0,1965-01-01 00:00:00+00:00,Gemini,36,36
1,1965-01-01 00:00:00+00:00,Voskhod,12,12
2,1966-01-01 00:00:00+00:00,Gemini,720,756
3,1969-01-01 00:00:00+00:00,Apollo,707,707
4,1969-01-01 00:00:00+00:00,Soyuz,37,37
...,...,...,...,...
63,2010-01-01 00:00:00+00:00,Space Shuttle,3591,61140
64,2011-01-01 00:00:00+00:00,ISS,1388,17901
65,2011-01-01 00:00:00+00:00,Space Shuttle,2492,63632
66,2012-01-01 00:00:00+00:00,ISS,2009,19910
